In [8]:
import os
# Create necessary directories in Colab for train and val splits
os.makedirs('images/train', exist_ok=True)
os.makedirs('images/val', exist_ok=True)
os.makedirs('labels/train', exist_ok=True)
os.makedirs('labels/val', exist_ok=True)

In [5]:
!pip install xmltodict
!pip install tqdm


  Using cached xmltodict-0.13.0-py2.py3-none-any.whl.metadata (7.7 kB)
Using cached xmltodict-0.13.0-py2.py3-none-any.whl (10.0 kB)


In [9]:
import os
import xml.etree.ElementTree as ET
from glob import glob
import random
from shutil import copyfile
from tqdm.notebook import tqdm  # Use tqdm.notebook for Jupyter Notebook compatibility

# Load class names from classes.txt
with open('classes.txt') as f:
    classes = f.read().strip().split()

# Helper function to convert bbox
def convert_bbox(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

# Helper function to convert annotation
def convert_annotation(xml_path, output_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    with open(output_path, 'w') as out_file:
        for obj in root.iter('object'):
            difficult = obj.find('difficult')
            if difficult is not None and difficult.text == '1':
                continue
            cls = obj.find('name').text
            if cls not in classes:
                continue
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
                 float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            bbox = convert_bbox((w, h), b)
            out_file.write(f"{cls_id} " + " ".join([f"{a:.6f}" for a in bbox]) + '\n')

# Create directories for YOLO formatted dataset
os.makedirs('images/train', exist_ok=True)
os.makedirs('images/val', exist_ok=True)
os.makedirs('labels/train', exist_ok=True)
os.makedirs('labels/val', exist_ok=True)

# Split dataset into train and val sets
image_files = glob(os.path.join('images', '*.jpg'))
random.shuffle(image_files)
train_ratio = 0.8
train_size = int(train_ratio * len(image_files))
train_files = image_files[:train_size]
val_files = image_files[train_size:]

# Copy and convert annotations for training set
for img_file in tqdm(train_files, desc='Processing training files'):
    base_name = os.path.basename(img_file)
    copyfile(img_file, os.path.join('images/train', base_name))
    xml_file = os.path.join('labels', base_name.replace('.jpg', '.xml'))
    if os.path.exists(xml_file):
        convert_annotation(xml_file, os.path.join('labels/train', base_name.replace('.jpg', '.txt')))
    else:
        print(f"Warning: {xml_file} not found.")

# Copy and convert annotations for validation set
for img_file in tqdm(val_files, desc='Processing validation files'):
    base_name = os.path.basename(img_file)
    copyfile(img_file, os.path.join('images/val', base_name))
    xml_file = os.path.join('labels', base_name.replace('.jpg', '.xml'))
    if os.path.exists(xml_file):
        convert_annotation(xml_file, os.path.join('labels/val', base_name.replace('.jpg', '.txt')))
    else:
        print(f"Warning: {xml_file} not found.")


Processing training files:   0%|          | 0/332 [00:00<?, ?it/s]

Processing validation files:   0%|          | 0/84 [00:00<?, ?it/s]

In [10]:
!pip install torch torchvision torchaudio
!pip install ultralytics


  Using cached torch-2.3.0-cp310-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached torchvision-0.18.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached filelock-3.14.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.3.0-cp310-none-macosx_11_0_arm64.whl (61.0 MB)
Using cached torchvision-0.18.0-cp310-cp310-macosx_11_0_arm64.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.5 MB/s eta 0:00:0000:0100:010m
Using cached filelock-3.14.0-py3-none-any.whl (12 kB)
Using cached networkx-3.3-py3-none-any.whl (1.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 6.2 MB/s eta 0:00:0000:0100:01m
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 239.0 kB/s eta 0:00:0000:0100:01
  Using cached opencv_python-4.9.0.80-cp37-abi3-macosx_11_0_arm64.whl.metadata

In [11]:
import yaml

# Read the classes from classes.txt
with open('classes.txt', 'r') as file:
    classes = [line.strip() for line in file]

# Filter for 'person' class
classes = [cls for cls in classes if cls == 'person']

# Define the dataset configuration
data = {
    'train': 'images/train',   # Path to training images
    'val': 'images/val',       # Path to validation images
    'nc': len(classes),         # Number of classes
    'names': classes           # List of class names
}

# Save to a .yaml file
with open('data.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)


In [23]:
from ultralytics import YOLO
import os

# Initialize YOLOv8 model
model = YOLO('yolov8n.pt')  # Load a pre-trained model or use 'yolov8s.pt', 'yolov8m.pt', etc.
yaml_path = 'data.yaml'

# Train the model using the YAML configuration file
model.train(data=yaml_path, epochs=50, imgsz=640)


# Save the trained model

model.save()


Ultralytics YOLOv8.2.25 🚀 Python-3.10.14 torch-2.3.0 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format

train: Scanning /Users/aryanmathur/Desktop/SYOOK/datasets/labels/train.cache... 

train: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/images/train/-1532-_png_jpg.rf.08a5b6985f24bfe7efefdb45c04469c2.jpg: ignoring corrupt image/label: Label class 7 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/images/train/-1579-_png_jpg.rf.c8f91ec3791bf03ccf9eca6c29f62aec.jpg: ignoring corrupt image/label: Label class 7 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/images/train/-1597-_png_jpg.rf.3bd5df66feaa51e0d65197b4acaf356f.jpg: ignoring corrupt image/label: Label class 7 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/images/train/-1817-_png_jpg.rf.0c0c9d7ee4b875c6ad49937fc72182f6.jpg: ignoring corrupt image/label: Label class 6 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/imag


val: Scanning /Users/aryanmathur/Desktop/SYOOK/datasets/labels/val.cache... 204 

val: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/images/val/-1532-_png_jpg.rf.08a5b6985f24bfe7efefdb45c04469c2.jpg: ignoring corrupt image/label: Label class 7 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/images/val/-1579-_png_jpg.rf.c8f91ec3791bf03ccf9eca6c29f62aec.jpg: ignoring corrupt image/label: Label class 7 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/images/val/-1817-_png_jpg.rf.0c0c9d7ee4b875c6ad49937fc72182f6.jpg: ignoring corrupt image/label: Label class 6 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/images/val/-184-_png_jpg.rf.b02963998a79b9ad5079f57b65130bc2.jpg: ignoring corrupt image/label: Label class 5 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/images/val/-1943-_png_j

...

      50/50         0G     0.5311     0.7726     0.9026         80        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all         38        175      0.975      0.944      0.989      0.875



50 epochs completed in 1.556 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.3MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.2.25 🚀 Python-3.10.14 torch-2.3.0 CPU (Apple M1)
Model summary (fused): 168 layers, 3007598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all         38        175      0.936      0.959      0.986      0.876
                person         38        105      0.962       0.96      0.989      0.916
              hard-hat         38         70      0.911      0.957      0.982      0.836
Speed: 2.0ms preprocess, 392.9ms inference, 0.0ms loss, 6.3ms postprocess per image
Results saved to runs/detect/train3


In [26]:
# Define the content for the YAML file
yaml_content = """
train: images/train
val: images/val

names:
  0: person
"""

# Specify the file name
file_name = 'person_data.yaml'

# Write the content to the YAML file
with open(file_name, 'w') as file:
    file.write(yaml_content)

print(f'{file_name} has been created with the specified content.')


person_data.yaml has been created with the specified content.


In [42]:
import os
import cv2
from ultralytics import YOLO

# Load the trained YOLOv8 model for person detection
person_model = YOLO('saved_model.pt')

# Directories
original_images_dir = 'images'  # Directory with original images
cropped_images_train_dir = 'cropped_images/train'
cropped_images_val_dir = 'cropped_images/val'
labels_train_dir = 'labels/train'  # Directory with original training labels
labels_val_dir = 'labels/val'  # Directory with original validation labels

# Create directories if they don't exist
os.makedirs(cropped_images_train_dir, exist_ok=True)
os.makedirs(cropped_images_val_dir, exist_ok=True)

# Function to crop persons from images and save labels
def crop_persons_from_images(image_dir, save_dir, label_dir):
    for img_name in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_name)
        label_path = os.path.join(label_dir, os.path.splitext(img_name)[0] + '.txt')
        
        if not os.path.exists(label_path):
            print("Skip if label file does not exist for ", img_name)
            continue  # Skip if label file does not exist
        
        img = cv2.imread(img_path)
        results = person_model(img)[0]  # Access the first element of the results list

        for idx, bbox in enumerate(results.boxes.xyxy):
            x1, y1, x2, y2 = map(int, bbox)
            cropped_img = img[y1:y2, x1:x2]
            cropped_img_name = f"{os.path.splitext(img_name)[0]}_person_{idx}.jpg"
            cropped_img_path = os.path.join(save_dir, cropped_img_name)
            cv2.imwrite(cropped_img_path, cropped_img)
            
            # Generate labels for cropped image
            with open(label_path, 'r') as f:
                lines = f.readlines()
            
            cropped_label_path = os.path.join(save_dir, os.path.splitext(cropped_img_name)[0] + '.txt')
            with open(cropped_label_path, 'w') as f:
                for line in lines:
                    class_id, cx, cy, w, h = map(float, line.split())
                    abs_cx = cx * img.shape[1]
                    abs_cy = cy * img.shape[0]
                    abs_w = w * img.shape[1]
                    abs_h = h * img.shape[0]

                    # Check if the PPE item is within the person's bounding box
                    if x1 <= abs_cx <= x2 and y1 <= abs_cy <= y2:
                        new_cx = (abs_cx - x1) / (x2 - x1)
                        new_cy = (abs_cy - y1) / (y2 - y1)
                        new_w = abs_w / (x2 - x1)
                        new_h = abs_h / (y2 - y1)
                        f.write(f"{int(class_id)} {new_cx} {new_cy} {new_w} {new_h}\n")

# Crop persons from training and validation images and generate labels
crop_persons_from_images(os.path.join(original_images_dir, 'train'), cropped_images_train_dir, 'labels/train')
crop_persons_from_images(os.path.join(original_images_dir, 'val'), cropped_images_val_dir, 'labels/val')

print('Person images and labels cropped and saved.')



0: 640x640 5 persons, 1 car, 1 dog, 207.7ms
Speed: 11.1ms preprocess, 207.7ms inference, 15.8ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 5 persons, 109.0ms
Speed: 7.5ms preprocess, 109.0ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 5 persons, 96.9ms
Speed: 1.7ms preprocess, 96.9ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 persons, 106.5ms
Speed: 1.7ms preprocess, 106.5ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

0: 640x640 2 persons, 1 train, 162.2ms
Speed: 1.5ms preprocess, 162.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 162.5ms
Speed: 2.8ms preprocess, 162.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 7 persons, 1 refrigerator, 101.6ms
Speed: 2.7ms preprocess, 101.6ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

0: 640x640 1 person, 1 potted plant, 156.5ms
Speed: 1.6ms

In [29]:
# Define the content for the YAML file for PPE detection
yaml_content = """
train: cropped_images/train
val: cropped_images/val

names:
  0: hard-hat
  1: gloves
  2: mask
  3: glasses
  4: boots
  5: vest
  6: ppe-suit
  7: ear-protector
  8: safety-harness
"""

# Specify the file name
file_name = 'ppe_data.yaml'

# Write the content to the YAML file
with open(file_name, 'w') as file:
    file.write(yaml_content)

print(f'{file_name} has been created with the specified content.')


ppe_data.yaml has been created with the specified content.


In [45]:
from ultralytics import YOLO

# Load the YOLOv8 model for PPE detection
ppe_model = YOLO('yolov8n.pt')

# Train the model using the YAML configuration file for PPE detection
ppe_model.train(data='ppe_data.yaml', epochs=50, imgsz=640)

# Save the trained model
ppe_model.save('best_ppe.pt')


New https://pypi.org/project/ultralytics/8.2.26 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.25 🚀 Python-3.10.14 torch-2.3.0 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=ppe_data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sa

train: Scanning /Users/aryanmathur/Desktop/SYOOK/datasets/cropped_images/train.c

train: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/cropped_images/train/-1477-_png_jpg.rf.bac8d06edca64da17ced23797d0e2339_person_0.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0651]
train: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/cropped_images/train/-1579-_png_jpg.rf.c8f91ec3791bf03ccf9eca6c29f62aec_person_0.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0033]
train: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/cropped_images/train/-1579-_png_jpg.rf.c8f91ec3791bf03ccf9eca6c29f62aec_person_1.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0508]
train: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/cropped_images/train/-1597-_png_jpg.rf.3bd5df66feaa51e0d65197b4acaf356f_person_0.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.4264      1.0863]
train: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/d


val: Scanning /Users/aryanmathur/Desktop/SYOOK/datasets/cropped_images/val.cache

val: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/cropped_images/val/-2180-_png_jpg.rf.9d63bb305e7747d22fe9a196dcc5ce13_person_0.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0353]
val: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/cropped_images/val/-2180-_png_jpg.rf.9d63bb305e7747d22fe9a196dcc5ce13_person_3.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0492      1.0395]
val: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/cropped_images/val/-2180-_png_jpg.rf.9d63bb305e7747d22fe9a196dcc5ce13_person_4.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [          1]
val: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/cropped_images/val/-2180-_png_jpg.rf.9d63bb305e7747d22fe9a196dcc5ce13_person_5.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0943]
val: WARNING ⚠️ /Users/aryanmathur/Desktop/SYOOK/datasets/cropped_im

...

      50/50         0G     0.8243     0.8639      1.161          5        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all        117        223      0.473      0.526       0.48      0.292



50 epochs completed in 5.497 hours.
Optimizer stripped from runs/detect/train12/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train12/weights/best.pt, 6.2MB

Validating runs/detect/train12/weights/best.pt...
Ultralytics YOLOv8.2.25 🚀 Python-3.10.14 torch-2.3.0 CPU (Apple M1)
Model summary (fused): 168 layers, 3007403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all        117        223      0.528      0.612      0.558      0.319
              hard-hat        117         63      0.501      0.825      0.749      0.556
                gloves        117         65      0.823      0.785      0.867      0.562
                  mask        117         15          0          0    0.00554    0.00225
                  vest        117         68      0.478      0.647      0.614      0.331
              ppe-suit        117          5      0.366          1      0.677      0.249
         ear-protector        117          7          1      0.416      0.434      0.213
Speed: 1.5ms preprocess, 182.8ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/train12


In [46]:
print('PPE detection model trained and saved.')

PPE detection model trained and saved.


In [55]:
import os
import cv2
import argparse
import sys
from ultralytics import YOLO

def inference_and_draw(models, image_path, output_path):
    img = cv2.imread(image_path)
    
    # Check if the image was read correctly
    if img is None:
        print(f"Error reading image {image_path}. Skipping.")
        return
    
    for model in models:
        results = model(img)  # Get results from model
        detections = results[0]  # Get the first image's results
        
        for box in detections.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = model.names[int(box.cls[0])]
            conf = box.conf[0]
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f'{label} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    cv2.imwrite(output_path, img)

def main(input_dir, output_dir, person_det_model, ppe_detection_model):
    # Load the trained YOLOv8 models
    person_model = YOLO(person_det_model)
    ppe_model = YOLO(ppe_detection_model)

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Perform inference on all images in the input directory
    for img_name in os.listdir(input_dir):
        img_path = os.path.join(input_dir, img_name)
        output_path = os.path.join(output_dir, img_name)
        inference_and_draw([person_model, ppe_model], img_path, output_path)

    print('Inference completed and results saved.')

if __name__ == '__main__':
    if 'ipykernel_launcher' in sys.argv[0]:
        # Simulate command-line arguments for Jupyter Notebook
        sys.argv = [
            'inference.py',
            '--input_dir', 'images',
            '--output_dir', 'output_dir',
            '--person_det_model', 'saved_model.pt',
            '--ppe_detection_model', 'best_ppe.pt'
        ]
    
    parser = argparse.ArgumentParser(description='YOLOv8 Inference Script')
    parser.add_argument('--input_dir', type=str, required=True, help='Directory with input images')
    parser.add_argument('--output_dir', type=str, required=True, help='Directory to save output images')
    parser.add_argument('--person_det_model', type=str, required=True, help='Path to the person detection model')
    parser.add_argument('--ppe_detection_model', type=str, required=True, help='Path to the PPE detection model')
    
    args = parser.parse_args()
    
    main(args.input_dir, args.output_dir, args.person_det_model, args.ppe_detection_model)



0: 448x640 2 persons, 172.9ms
Speed: 24.3ms preprocess, 172.9ms inference, 13.8ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 persons, 107.2ms
Speed: 2.4ms preprocess, 107.2ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 persons, 105.9ms
Speed: 2.3ms preprocess, 105.9ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 boat, 101.3ms
Speed: 1.8ms preprocess, 101.3ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

0: 640x640 5 persons, 1 car, 1 dog, 155.2ms
Speed: 2.4ms preprocess, 155.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 persons, 1 car, 144.7ms
Speed: 1.9ms preprocess, 144.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 5 persons, 103.8ms
Speed: 2.4ms preprocess, 103.8ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 5 persons, 109.2ms
Speed: 2.1ms preprocess, 109.2ms inference, 0.